In [26]:
import sys
sys.path.append('./models/sixdrepnet/')
import argparse
import time
from pathlib import Path

import os
import copy
import cv2
import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.hub import load_state_dict_from_url
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box
from utils.plots import colors, plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

from models.sixdrepnet import model as sixdmodel
from easydict import EasyDict as edict



In [27]:
args = edict()
args.weights='./yolov7-tiny-face.pt'
args.source='./image.jpg'
args.img_size=416
args.conf_thres=0.25
args.iou_thres=0.45
args.device='0'
args.view_img=False
args.save_txt=False
args.save_txt_tidl=False
args.save_bin=False
args.save_conf=False
args.save_crop=False
args.classes=0
args.nosave=False
args.agnostic_nms=False
args.augment=False
args.update=False
args.project='runs/detect'
args.name='exp'
args.exist_ok=False
args.line_thickness=3
args.hide_labels=False
args.hide_conf=False
args.kpt_label=5
args.use_dof=True

opt = args

In [36]:
def detect(opt):
    source, weights, view_img, save_txt, imgsz, save_txt_tidl, kpt_label = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, opt.save_txt_tidl, opt.kpt_label
    save_img = not opt.nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    use_dof = opt.use_dof
    # Directories
    save_dir = increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok)  # increment run
    (save_dir / 'labels' if (save_txt or save_txt_tidl) else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(opt.device)
    half = device.type != 'cpu' and not save_txt_tidl  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    
    if use_dof:
        sixdofmodel = sixdmodel.SixDRepNet(backbone_name='RepVGG-B1g2',
                                backbone_file='',
                                deploy=True,
                                pretrained=False,
                                postprocess=False,
                                gpu_id=device)
        state_dict = load_state_dict_from_url("https://cloud.ovgu.de/s/Q67RnLDy6JKLRWm/download/6DRepNet_300W_LP_AFLW2000.pth")   
        sixdofmodel.eval()
        sixdofmodel.load_state_dict(state_dict)
        mean=torch.Tensor([0.485, 0.456, 0.406]).reshape(3,1,1).to(device)
        std=torch.Tensor([0.229, 0.224, 0.225]).reshape(3,1,1).to(device)
        sixdofmodel.to(device)

        
    stride = int(model.stride.max())  # model stride
    if isinstance(imgsz, (list,tuple)):
        assert len(imgsz) ==2; "height and width of image has to be specified"
        imgsz[0] = check_img_size(imgsz[0], s=stride)
        imgsz[1] = check_img_size(imgsz[1], s=stride)
    else:
        imgsz = check_img_size(imgsz, s=stride)  # check img_size
    names = model.module.names if hasattr(model, 'module') else model.names  # get class names
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=opt.augment)
        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms, kpt_label=kpt_label)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                if use_dof:
                    for det_idx, det_box in enumerate(det[:, :4]):
                        #img shape = (3,416,416) / det [lu_x, lu_y, rb_x, rb_y]
                        print(det_idx)
                        c1 = (int(det_box[0]), int(det_box[1]))
                        c2 = (int(det_box[2]), int(det_box[3]))
                        print(f"c1, c2 = {c1}, {c2}")
                        img_crop = img[:,:,c1[1]:c2[1], c1[0]:c2[0]]
                        img_resize = F.interpolate(img_crop, size=(224, 224), mode='bilinear', align_corners=False)
                        img_resize = (img_resize - mean) / std # normalize
                        print(f"img_resize's shape : {img_resize.shape}, type : {type(img_resize)}")
                        out = sixdofmodel(img_resize)
                        print(out)

                scale_coords(img.shape[2:], det[:, :4], im0.shape, kpt_label=False)
                scale_coords(img.shape[2:], det[:, 6:], im0.shape, kpt_label=kpt_label, step=3)

                # Print results
                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for det_index, (*xyxy, conf, cls) in enumerate(reversed(det[:,:6])):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or opt.save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if opt.hide_labels else (names[c] if opt.hide_conf else f'{names[c]} {conf:.2f}')
                        kpts = det[det_index, 6:]
                        plot_one_box(xyxy, im0, label=label, color=colors(c, True), line_thickness=opt.line_thickness, kpt_label=kpt_label, kpts=kpts, steps=3, orig_shape=im0.shape[:2])
                        if opt.save_crop:
                            save_one_box(xyxy, im0s, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)


                if save_txt_tidl:  # Write to file in tidl dump format
                    for *xyxy, conf, cls in det_tidl:
                        xyxy = torch.tensor(xyxy).view(-1).tolist()
                        line = (conf, cls,  *xyxy) if opt.save_conf else (cls, *xyxy)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_txt_tidl or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt or save_txt_tidl else ''
        print(f"Results saved to {save_dir}{s}")

    print(f'Done. ({time.time() - t0:.3f}s)')




In [37]:
detect(opt)

YOLOv5 � b4b8334 torch 1.13.0a0+d0d6b1f CUDA:0 (NVIDIA GeForce RTX 3060, 12049.875MB)

Fusing layers... 
Model Summary: 314 layers, 7826611 parameters, 0 gradients


image 1/1 /home/pdh/yolov7-face/image.jpg: torch.Size([1, 10647, 21])
0
c1, c2 = (161, 117), (187, 149)
img_resize's shape : torch.Size([1, 3, 224, 224]), type : <class 'torch.Tensor'>
tensor([[34.68848,  9.21260, 20.45169, -6.45441, 45.38958, -4.58429]], device='cuda:0', grad_fn=<AddmmBackward0>)
1
c1, c2 = (230, 156), (254, 187)
img_resize's shape : torch.Size([1, 3, 224, 224]), type : <class 'torch.Tensor'>
tensor([[ 6.83339,  0.54192,  5.58162, -0.72736, 11.59490, -1.99125]], device='cuda:0', grad_fn=<AddmmBackward0>)
2
c1, c2 = (315, 138), (339, 174)
img_resize's shape : torch.Size([1, 3, 224, 224]), type : <class 'torch.Tensor'>
tensor([[ 10.02235,  -5.21668, -20.28385,   0.56064,  24.21293,  -5.28454]], device='cuda:0', grad_fn=<AddmmBackward0>)
3
c1, c2 = (110, 115), (136, 156)
img_resize's shape : torch.Size([1, 3, 224, 224]), type : <class 'torch.Tensor'>
tensor([[ 0.39879, -0.03525,  1.64965, -0.62088,  5.19641,  0.12191]], device='cuda:0', grad_fn=<AddmmBackward0>)
416x416 